In [7]:
# %pip install -q translate
# %pip install -q underthesea==1.3.5a3
# %pip install -q underthesea[deep]
# %pip install -q pyvi
# %pip install -q langdetect
# %pip install -q googletrans==3.1.0a0
# %pip install -q peft
%pip install bitsandbytes
%pip install transformers
%pip install flash-attn
%pip install -U sentence-transformers
%pip install xformers

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached flash_attn-2.6.3.tar.gz (2.6 MB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [20 lines of output]
      fatal: not a git repository (or any of the parent directories): .git
      
      
      torch.__version__  = 2.4.0
      
      
      /tmp/pip-install-3e7yeyci/flash-attn_914243b39ddc426c96138a23ec073b8f/setup.py:95: UserWarning: flash_attn was requested, but nvcc was not found.  Are you sure your environment has nvcc available?  If you're installing within a container from https://hub.docker.com/r/pytorch/pytorch, only images whose names contain 'devel' will provide nvcc.
        warnings.warn(
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", lin

In [1]:
# %pip install -q faiss-cpu
%pip install -q git+https://github.com/openai/CLIP.git

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import torch
import clip
from PIL import Image
import faiss
import numpy as np
import json
import matplotlib.pyplot as plt
import math
import googletrans
import translate
import glob
import underthesea
import sys
import time
from tqdm import tqdm
from pyvi import ViUtils, ViTokenizer
from difflib import SequenceMatcher
from langdetect import detect
from pathlib import Path
import re
from transformers import BlipForConditionalGeneration


/home/duong/anaconda3/envs/aic/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
ROOT = Path(os.getcwd()).resolve()

# Add ROOT to sys.path
sys.path.append(str(ROOT))

# Determine the working directory
if len(ROOT.parents) > 1:
    WORK_DIR = ROOT.parents[0]
else:
    WORK_DIR = ROOT  # Fallback to ROOT if it doesn't have enough parents

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model, preprocess = clip.load("ViT-B/32", device=device)

paths = f"{WORK_DIR}/data/keyframes"
des_path =  f"{WORK_DIR}/working//dicts/npy_blip"
os.makedirs(des_path, exist_ok=True)
print(WORK_DIR)

if len(WORK_DIR.parents) > 1:
    WORK_DIR2 = WORK_DIR.parents[0]
else:
    WORK_DIR2 = WORK_DIR  # Fallback to ROOT if it doesn't have enough parents
path2 = f"{WORK_DIR2}/data/keyframes"
print(path2)


/home/duong/Documents/AIC2024_UTE_AI_Unknown/src
/home/duong/Documents/AIC2024_UTE_AI_Unknown/data/keyframes


In [3]:
from transformers import DPRContextEncoder, AutoProcessor, DPRContextEncoderTokenizer, BlipModel,TrOCRProcessor, VisionEncoderDecoderModel, ViTFeatureExtractor, AutoTokenizer, BitsAndBytesConfig, BlipForConditionalGeneration
#from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training
from sentence_transformers import SentenceTransformer


# tokenizer = DPRContextEncoderTokenizer.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base', use_fast=False)
# embedding_model = DPRContextEncoder.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base').to(device)

embedding_model= SentenceTransformer("nomic-ai/nomic-embed-text-v1", trust_remote_code=True)

/home/duong/anaconda3/envs/aic/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/duong/.cache/huggingface/modules/transformers_modules/nomic-ai/nomic-bert-2048/e55a7d4324f65581af5f483e830b80f34680e8ff/modeling_hf_nomic_bert.py:95: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless the

In [7]:
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
#Another model for Blip
# https://huggingface.co/models?sort=trending&search=blip
processor = AutoProcessor.from_pretrained("Salesforce/blip-image-captioning-base")

NameError: name 'AutoProcessor' is not defined

In [9]:
def blip_image(image_path):
  image = Image.open(image_path)
  inputs = processor(images=image, return_tensors="pt").to(device)
  pixel_values = inputs.pixel_values

  generated_ids = model.generate(pixel_values=pixel_values, max_length=50)
  generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
  return generated_caption

In [10]:

for keyframe in tqdm(os.listdir(path2)):
  path_keyframe = os.path.join(path2,keyframe)
  video_paths = sorted(glob.glob(f"{path_keyframe}/*/"))
  video_paths = ['/'.join(i.split('/')[:-1]) for i in video_paths]

  start_time = time.time()
  for vd_path in video_paths:

    re_feats = []
    keyframe_paths = glob.glob(f'{vd_path}/*.jpg')
    keyframe_paths = sorted(keyframe_paths, key=lambda x : x.split('/')[-1].replace('.jpg',''))

    for keyframe_path in tqdm(keyframe_paths):


      #text = ocr_image(keyframe_path)

      #//////////////////////////////////
      text = blip_image(keyframe_path)
      #//////////////////////////////////
      if detect(text) == 'vi' :
        text = Translation(text)

      # Convert text to embedding vector
      #embedding = embedding_model(**tokenizer(text, return_tensors="pt", padding=True, truncation=True,max_length=512, add_special_tokens = True)).pooler_output.detach().numpy()
      embeddings = embedding_model.encode(text)
      # Append embedding to re_feats list
      re_feats.append(embeddings)

    name_npy = vd_path.split('/')[-1]

    # Construct output file path
    outfile = os.path.join(des_path, f'{name_npy}.npy')

    # Ensure the directory exists before saving
    os.makedirs(des_path, exist_ok=True)
    np.save(outfile, re_feats)

    print(f"Processed {vd_path} in {time.time() - start_time} seconds")


100%|██████████| 235/235 [04:10<00:00,  1.07s/it]


Processed /home/duong/Documents/AIC2024_UTE_AI_Unknown/data/keyframes/Keyframes_L30/L30_V001 in 250.8251929283142 seconds


 50%|█████     | 1/2 [09:00<09:00, 540.02s/it]

Processed /home/duong/Documents/AIC2024_UTE_AI_Unknown/data/keyframes/Keyframes_L30/L30_V002 in 540.0229551792145 seconds


100%|██████████| 241/241 [03:57<00:00,  1.01it/s]


Processed /home/duong/Documents/AIC2024_UTE_AI_Unknown/data/keyframes/Keyframes_L31/L31_V001 in 237.96122980117798 seconds


100%|██████████| 2/2 [16:51<00:00, 505.71s/it]

Processed /home/duong/Documents/AIC2024_UTE_AI_Unknown/data/keyframes/Keyframes_L31/L31_V002 in 471.39397144317627 seconds


In [6]:
feature_shape = 384


def write_bin_file_ocr(bin_path: str, npy_path: str, method='cosine'):
    if method in 'L2':
      index = faiss.IndexFlatL2(feature_shape)
    elif method in 'cosine':
      index = faiss.IndexFlatIP(feature_shape)
    else:
      assert f"{method} not supported"
    npy_files = glob.glob(os.path.join(npy_path, "*.npy"))
    npy_files_sorted = sorted(npy_files)

    for npy_file in npy_files_sorted:
        feats = np.load(npy_file)
        print(f"Loaded {npy_file}, shape: {feats.shape}")


        # Convert to float32 and reshape to match feature_shape
        feats = feats.astype(np.float32)
        feats = feats.reshape(-1, feats.shape[-1])

        # Resize or trim feats_normalized to match feature_shape if necessary
        if feats.shape[1] != feature_shape:
            feats = feats[:, :feature_shape]

        assert feats.shape[1] == feature_shape, \
            f"Query features dimension {feats.shape[1]} do not match index dimension {feature_shape}"

        # Add to Faiss index
        index.add(feats)

    # Write the Faiss index to disk
    faiss.write_index(index, os.path.join(bin_path, f"faiss_BLIP_{method}.bin"))
    print(f'Saved {os.path.join(bin_path, f"faiss_BLIP_{method}.bin")}')


# write ocr
write_bin_file_ocr(bin_path=f"{WORK_DIR}/working/dicts/bin_blip", npy_path=f"{WORK_DIR}/working/dicts/npy_blip")



Loaded /home/duong/Documents/AIC2024_UTE_AI_Unknown/src/working/dicts/npy_blip/L30_V001.npy, shape: (235, 768)
Loaded /home/duong/Documents/AIC2024_UTE_AI_Unknown/src/working/dicts/npy_blip/L30_V002.npy, shape: (273, 768)
Loaded /home/duong/Documents/AIC2024_UTE_AI_Unknown/src/working/dicts/npy_blip/L31_V001.npy, shape: (241, 768)
Loaded /home/duong/Documents/AIC2024_UTE_AI_Unknown/src/working/dicts/npy_blip/L31_V002.npy, shape: (244, 768)
Saved /home/duong/Documents/AIC2024_UTE_AI_Unknown/src/working/dicts/bin_blip/faiss_BLIP_cosine.bin
